In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import scipy.stats as stats
from IPython.display import Markdown, display
#from python_scripts.pricingprocessing import *
import pickle


In [2]:
comparison_set, bad_set = load_comparison_pickle()
df = pull_pricing_draft(comparison_set, bad_set, limit=None)


total entries: 14085
entries over power 2: 1108


In [3]:
df['created'] = pd.to_datetime(df['created'], unit='s')
df['price'] = df['selected_text'].map(lambda x: int(x[1:].strip()))

In [3]:
backup_df = df.copy()

In [2]:
with open('/home/erin/PycharmProjects/HardwareScrape/data/df.P', 'rb') as pickle_file:
    df = pickle.load(pickle_file)

In [5]:
def drop_multi_item_posts(df):
    df['split_num'] = df['title_select'].str.split(',')
    df['split_num'] = df['split_num'].map(lambda x: len(x))
    df.loc[df['split_num'] != 1]
    df = df.loc[df['split_num'] == 1].copy()
    df = df.drop(['split_num'], axis=1)
    return df
df = drop_multi_item_posts(df)

In [6]:
def model_identification(df):
    df['ti'] = df['title_select'].str.contains('ti')
    
model_identification(df)

In [7]:
df['model'] = df['title_select'].str.extract(r'(\d{3,4})')

In [59]:
def drop_undesirable_data(df):
    df = df.loc[df['full_text'].str.contains('laptop') == False]
    df = df.loc[df['title_select'].str.contains('laptop') == False]
    df = df.loc[df['post_id'] != 'bl70vy']
    df = df.loc[df['model'].isnull()==False]
    
    df = df.loc[df['model'] != '120'] #MSI Laptop
    df = df.loc[df['model'] != '2400'] #Ryzen 2400g
    df = df.loc[df['model'] != '290'] #multi item post
    df = df.loc[df['model'] != '299'] #multi item
    df = df.loc[df['model'] != '3570']
    df = df.loc[df['model'] != '480']#amd rx 480
    df = df.loc[df['model'] != '503']
    df = df.loc[df['model'] != '6600'] #full pc sale
    df = df.loc[df['model'] != '6700'] #full pc sale
    df = df.loc[df['model'] != '7700'] # full pc sale
    df = df.loc[df['model'] != '870'] # full pc sale

    
    return df
df = drop_undesirable_data(df)

In [130]:
post_id = 'cmjrtu'
list(df.loc[df['post_id']==post_id, 'full_text']) 
# df.loc[df['post_id']==post_id, 'price'] = 1900.0

["I've  put this off for a while, I upgraded my GPU to something better and put  this in my home server about a week ago so it wouldn't collect dust,  but it's insane overkill for what my home server does as it handles  mostly CPU bound tasks.\n\nI gamed  using this GPU pretty moderately for about 2 months before upgrading. It  has some wear and tear from a case mod I did when I got it, and the fan  shroud on the bottom of the GPU was dremeled a little bit to fit a  cable through, it's nothing you would see if the GPU is mounted  horizontally.\n\n~~I want $450 shipped.~~\n\nSOLD: Pending payment\n\nTimestamp: [https://imgur.com/a/mIPnS88](https://imgur.com/a/mIPnS88)"]

In [131]:
with open('/home/erin/PycharmProjects/HardwareScrape/data/df.P', 'wb') as pickle_file:
    pickle.dump(df, pickle_file)

In [128]:
df.loc[(df['model'] == '2080' )& (df['price'] < 500)].sort_values('price')

,title_select,selected_text,full_text,full_title,location_tag,post_id,author_id,author_trades,created,price,ti,model
180,gigabyte rtx 2080 gaming oc white,$450,"I've put this off for a while, I upgraded my ...",[USA-NC] [H] Gigabyte RTX 2080 Gaming OC White...,[USA-NC],cmjrtu,t2_rv2nw,Trades: 5,2019-08-06 09:00:17,450.0,False,2080
124,gigabyte rtx 2080 gaming oc,$465,[Timestamp](https://imgur.com/a/zJx1jHo)\n\nLo...,[USA-CA][H] Gigabyte RTX 2080 Gaming OC [W] Lo...,[USA-CA],crepqz,t2_4noia19,Trades: 6,2019-08-17 08:05:47,465.0,False,2080


In [5]:
model_price = df.groupby(['model', 'ti']).agg('mean').sort_values('model')
# df.head()

In [ ]:
np.mean(df['title_select'].str.len())

In [ ]:
df.loc[df['title_select'].str.len() > 80]


In [19]:
df.loc[df['ti'] == True]


,title_select,selected_text,full_text,full_title,location_tag,post_id,author_id,author_trades,created,price,ti,model
7,asus rog strix gtx 1080 ti oc edition,$550,This is the OC edition ROG Strix card from ASU...,[USA-CA] [H] ASUS ROG Strix GTX 1080 Ti OC Edi...,[USA-CA],d1jhqh,t2_48lin,Trades: 4,2019-09-09 08:36:38,550,True,1080
8,"i7-4790k, msi 980 ti, gigabyte ga-z97x-sli, 2...",$675,Hello! First time here.\n\nTimestamps: https:/...,"[USA-NY] [H] i7-4790K, MSI 980 Ti, Gigabyte GA...",[USA-NY],d1fdjs,t2_ragfo,None,2019-09-09 03:11:21,675,True,4790
10,asus rog strix 1080 ti,$560,$560 shipped\n\n[timestamps](https://imgur.com...,[USA-CA] [H] ASUS ROG STRIX 1080 Ti [W] Paypal...,[USA-CA],d12iqq,t2_algqs,Trades: 7,2019-09-08 06:16:06,560,True,1080
11,nvidia rtx 2080 ti,$900,I have a Zotac Gaming Nvidia RTX 2080 ti for s...,"[USA-MD] [H] Nvidia RTX 2080 ti [W] PayPal, Lo...",[USA-MD],d0xku8,t2_31plw9ps,Trades: 13,2019-09-07 23:45:48,900,True,2080
16,asus strix geforce gtx 1070 8 gb oc edition,$250,GPU is being sold due to an upgrade. Used sole...,[USA - NJ] [H] ASUS STRIX GeForce GTX 1070 8 G...,[USA - NJ],d09uxx,t2_g9cib,None,2019-09-06 09:24:17,250,True,1070
...,...,...,...,...,...,...,...,...,...,...,...,...
1091,nvidia geforce gtx 1070 founders edition,$230,Card is in great condition besides a small scr...,[USA-VA] [H] Nvidia Geforce GTX 1070 Founders ...,[USA-VA],acbhli,t2_9zeer,Trades: 3,2019-01-04 07:08:23,230,True,1070
1094,evga gtx 980 ti hybrid,$200,Pic: [https://imgur.com/lewGLLb](https://imgur...,[USA-TX][H] EVGA GTX 980 Ti Hybrid [W] Paypal,[USA-TX],ac32fm,t2_5fikj,None,2019-01-03 14:49:31,200,True,980
1099,asus strix 1080 ti oc,$600,Happy new year!\n\nToo big to fit in my ITX ca...,[USA-TX] [H] Asus Strix 1080 Ti OC [W] Local C...,[USA-TX],abwlua,t2_17ec5i,Trades: 30,2019-01-03 03:19:37,600,True,1080
1100,sc geforce gtx 1050 ti,$110,Asking $110 or best offer \nUsed lightly for a...,[USA-FL] [H] SC GeForce GTX 1050 Ti [W] Cash l...,[USA-FL],abvvgu,t2_142ti9,None,2019-01-03 02:08:28,110,True,1050


In [6]:
with pd.option_context("display.max_rows", 1000):
    display(model_price)


price
model ti               
1030  False   63.500000
1050  False   75.000000
      True    97.806452
1060  False  142.471698
      True   122.500000
1070  False  227.376238
      True   276.175000
1080  False  336.672897
      True   503.684211
120   False   60.000000
1650  False  133.333333
1660  True   217.500000
      False  170.000000
2060  False  308.395349
      True   306.666667
2070  False  422.672131
      True   492.142857
2080  False  645.986486
      True   586.560000
2400  False  380.000000
      True   600.000000
290   True   100.000000
299   True   500.000000
3570  False   70.000000
460   False   50.000000
4790  False  175.000000
480   False  130.000000
503   False  800.000000
560   True    44.000000
570   False   85.000000
      True    90.000000
580   False   92.500000
650   False   35.000000
660   False   65.000000
      True    50.000000
6600  False  600.000000
6700  True   600.000000
720   False  620.000000
750   True    46.666667
760   False   51.666667
770   False   65.833333
7700  True   300.000000
780   False  115.000000
      True   120.000000
870   False  550.000000
950   False   56.428571
960   False   72.500000
      True    75.000000
970   False  104.563636
980   False  180.576923
      True   215.131579

In [18]:
df.loc[df['model']=='290']

,title_select,selected_text,full_text,full_title,location_tag,post_id,author_id,author_trades,created,price,ti,model
807,r9 290x w/accel-x3 | gtx 980ti 6gb | r9 390x ...,$100,Items For Sale...\n\n# Prices Revised (per sug...,[USA-CT] [H] R9 290X w/ACCEL-X3 | GTX 980TI 6G...,[USA-CT],az7bh9,t2_pba7d,Trades: 26,2019-03-10 03:50:53,100,True,290


In [7]:
test_price_df = pd.DataFrame()

In [12]:
test_price_df['model'] = model_price['model',].map

In [19]:
model_price = model_price.reset_index()

In [29]:
test_price_df['model'] = model_price['model'].loc[model_price['ti'] == False]

In [44]:
backup_test_prices = test_price_df.copy()

In [42]:

ti_model_series = model_price.reset_index()['model'].loc[model_price['ti'] == True].map(lambda x: x+'ti')

In [59]:
test_price_df['model'] = pd.Series.append(test_price_df['model'], ti_model_series, ignore_index=True)

In [60]:
test_price_df


,model
2,1070
4,1660
6,2070
8,2400
11,560
14,6700
16,7700
18,960
20,1050ti
21,1060ti


In [32]:
test_price_df['model'] = model_price['model']

In [38]:
test_price_df

,model
2,1050
4,1060
6,1070
8,1080
11,1660
14,2060
16,2070
18,2080
20,2400
21,290


graph plan:
* Count of each model histogram/bar chart
    * need to figure out how to separate model from ti models
* price/performance used
    * this needs pricing data from wiki? MSRP
    * performance data from website
        * this will take time but is just copy pasting values.


In [2]:
from pymongo_db import db

collection = db['frontend']

df = pd.DataFrame.from_records(collection.find())


In [45]:
x = df[['model','is_ti', 'price']].groupby(['model', 'is_ti']).agg('mean').reset_index()

In [97]:
models = np.unique(x.model)
test = pd.DataFrame({'model':models})

In [36]:
test.loc[:, 'non_ti_price'] = test.model.map(lambda model: (x.price.at[x.loc[(x.is_ti==False) & (x.model == model)]]))
# test.loc[:, 'ti_price'] = test.model.map(lambda model: (x.price.at[(x.is_ti==True) & (x.model == model)]))

ValueError: Invalid call for scalar access (getting)!

In [95]:
# x.price.loc[(x.model==760) & (x.is_ti==True)]
ti_toggle = True
test.model.map(lambda model:x.price.loc[(x.model==model)&(x.is_ti==ti_toggle)].values[0] if (x.price.loc[(x.model==model)&(x.is_ti==ti_toggle)]).size !=0 else 0)

0       0.000000
1     110.000000
2       0.000000
3       0.000000
4       0.000000
5     243.478261
6       0.000000
7     105.000000
8       0.000000
9     295.076923
10    512.812500
11      0.000000
12    217.500000
13      0.000000
14      0.000000
15    538.386364
Name: model, dtype: float64

In [112]:
y = test.merge(x[['model', 'price']].loc[x.is_ti==False], on='model').rename(columns={'price':"non_ti_price"})
y = y.merge(x[['model', 'price']].loc[x.is_ti==True], on='model', how='outer').rename(columns={'price':"ti_price"})
y = y.fillna(0.0)

,model,non_ti_price,ti_price
0,760,69.285714,0.000000
1,780,106.666667,110.000000
2,950,56.428571,0.000000
3,960,82.121212,0.000000
4,970,105.183333,0.000000
5,980,194.333333,243.478261
6,1030,63.500000,0.000000
7,1050,108.037037,105.000000
8,1060,155.101695,0.000000
9,1070,243.720930,295.076923


In [117]:
y.to_json(orient='records')

'[{"model":760,"non_ti_price":69.2857142857,"ti_price":0.0},{"model":780,"non_ti_price":106.6666666667,"ti_price":110.0},{"model":950,"non_ti_price":56.4285714286,"ti_price":0.0},{"model":960,"non_ti_price":82.1212121212,"ti_price":0.0},{"model":970,"non_ti_price":105.1833333333,"ti_price":0.0},{"model":980,"non_ti_price":194.3333333333,"ti_price":243.4782608696},{"model":1030,"non_ti_price":63.5,"ti_price":0.0},{"model":1050,"non_ti_price":108.037037037,"ti_price":105.0},{"model":1060,"non_ti_price":155.1016949153,"ti_price":0.0},{"model":1070,"non_ti_price":243.7209302326,"ti_price":295.0769230769},{"model":1080,"non_ti_price":351.6824324324,"ti_price":512.8125},{"model":1650,"non_ti_price":133.3333333333,"ti_price":0.0},{"model":1660,"non_ti_price":170.0,"ti_price":217.5},{"model":2060,"non_ti_price":309.170212766,"ti_price":0.0},{"model":2070,"non_ti_price":431.7777777778,"ti_price":0.0},{"model":2080,"non_ti_price":639.1075268817,"ti_price":538.3863636364}]'

In [22]:
df.loc[:, 'datestring'] = (pd.to_datetime(df.created, unit='s').dt.strftime('%m-%d-%y'))
non_ti = df[['post_id', 'datestring', 'created', 'price']].loc[(df.model==980) & (df.is_ti == False)].head(5).to_json(orient='records')
ti = df[['post_id', 'datestring', 'created', 'price']].loc[(df.model==980) & (df.is_ti == True)].head(5).to_json(orient='records')


In [23]:
testing_json_bit = ('[' + non_ti + ', ' + ti + ']')


In [21]:
# df.datetime.dt.strftime()
int('blah')

ValueError: invalid literal for int() with base 10: 'blah'